# Task 1: Data Handling

In [1]:
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Optional: For better aesthetics
sns.set(style="dark")
sns.set_palette("tab10")
%matplotlib inline

# Load the datasets
data_changping = pd.read_csv("PRSA_Data_Changping_20130301-20170228.csv")
data_huairou = pd.read_csv("PRSA_Data_Huairou_20130301-20170228.csv")

# Concatenate the two datasets
data = pd.concat([data_changping, data_huairou], ignore_index=True)

# Show the first few rows
data.head()

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
0,1,2013,3,1,0,3.0,6.0,13.0,7.0,300.0,85.0,-2.3,1020.8,-19.7,0.0,E,0.5,Changping
1,2,2013,3,1,1,3.0,3.0,6.0,6.0,300.0,85.0,-2.5,1021.3,-19.0,0.0,ENE,0.7,Changping
2,3,2013,3,1,2,3.0,3.0,22.0,13.0,400.0,74.0,-3.0,1021.3,-19.9,0.0,ENE,0.2,Changping
3,4,2013,3,1,3,3.0,6.0,12.0,8.0,300.0,81.0,-3.6,1021.8,-19.1,0.0,NNE,1.0,Changping
4,5,2013,3,1,4,3.0,3.0,14.0,8.0,300.0,81.0,-3.5,1022.3,-19.4,0.0,N,2.1,Changping


# Task 2: EDA

## Task 2a: Fundamental Data Understanding

In [2]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
No,70128.0,17532.500000,10122.177085,1.0000,8766.75,17532.5,26298.25,35064.0
year,70128.0,2014.662560,1.177205,2013.0000,2014.00,2015.0,2016.00,2017.0
month,70128.0,6.522930,3.448728,1.0000,4.00,7.0,10.00,12.0
day,70128.0,15.729637,8.800155,1.0000,8.00,16.0,23.00,31.0
hour,70128.0,11.500000,6.922236,0.0000,5.75,11.5,17.25,23.0
PM2.5,68401.0,70.364983,71.782734,2.0000,18.00,46.0,99.00,882.0
PM10,68769.0,93.074782,83.380418,2.0000,31.00,70.0,131.00,999.0
SO2,68520.0,13.547517,20.018705,0.2856,2.00,5.0,16.00,315.0
NO2,67822.0,38.423399,28.668603,1.0265,17.00,31.0,54.00,231.0
CO,67185.0,1087.332351,1008.029758,100.0000,500.00,800.0,1400.00,10000.0


In [ ]:
data.describe(include='object').T

In [3]:
data.shape

(70128, 18)

In [4]:
data.isna().sum()

No            0
year          0
month         0
day           0
hour          0
PM2.5      1727
PM10       1359
SO2        1608
NO2        2306
CO         2943
O3         1755
TEMP        104
PRES        103
DEWP        106
RAIN        106
wd          442
WSPM         92
station       0
dtype: int64

In [5]:
# Check percentage of missing values
missing_percent = data.isnull().sum() / len(data) * 100
print(missing_percent)

No         0.000000
year       0.000000
month      0.000000
day        0.000000
hour       0.000000
PM2.5      2.462640
PM10       1.937885
SO2        2.292950
NO2        3.288273
CO         4.196612
O3         2.502567
TEMP       0.148300
PRES       0.146874
DEWP       0.151152
RAIN       0.151152
wd         0.630276
WSPM       0.131189
station    0.000000
dtype: float64


In [6]:
data.duplicated().sum()

np.int64(0)

In [7]:
import pandas as pd

def count_outliers(data, columns):
    outlier_counts = {}

    for column in columns:
        # Calculate Q1 (25th percentile) and Q3 (75th percentile)
        Q1 = data[column].quantile(0.25)
        Q3 = data[column].quantile(0.75)

        # Calculate the IQR (Interquartile Range)
        IQR = Q3 - Q1

        # Calculate lower and upper bounds
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Count the number of outliers (values outside the bounds)
        outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
        outlier_counts[column] = outliers.shape[0]

    # Convert the outlier counts into a DataFrame
    outlier_df = pd.DataFrame(list(outlier_counts.items()), columns=['Pollutant', 'Outlier Count'])
    return outlier_df

# Specify the columns (pollutants)
pollutants = ['PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3']

# Get the count of outliers per column
outliers_per_column = count_outliers(data, pollutants)

# Print out the DataFrame with the number of outliers per column
outliers_per_column

,Pollutant,Outlier Count
0,PM2.5,3347
1,PM10,2358
2,SO2,6581
3,NO2,1677
4,CO,4036
5,O3,3170


## Task 2b: Data Preprocessing

In [8]:
data = data.dropna()

In [9]:
data['datetime'] = pd.to_datetime(data[['year', 'month', 'day', 'hour']])

cols = ['datetime'] + [col for col in data.columns if col != 'datetime']
data = data[cols]

data = data.drop(columns=['No', 'year', 'month', 'day', 'hour'])

In [10]:
data.head()

,datetime,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
0,2013-03-01 00:00:00,3.0,6.0,13.0,7.0,300.0,85.0,-2.3,1020.8,-19.7,0.0,E,0.5,Changping
1,2013-03-01 01:00:00,3.0,3.0,6.0,6.0,300.0,85.0,-2.5,1021.3,-19.0,0.0,ENE,0.7,Changping
2,2013-03-01 02:00:00,3.0,3.0,22.0,13.0,400.0,74.0,-3.0,1021.3,-19.9,0.0,ENE,0.2,Changping
3,2013-03-01 03:00:00,3.0,6.0,12.0,8.0,300.0,81.0,-3.6,1021.8,-19.1,0.0,NNE,1.0,Changping
4,2013-03-01 04:00:00,3.0,3.0,14.0,8.0,300.0,81.0,-3.5,1022.3,-19.4,0.0,N,2.1,Changping


## Task 2c: Statistical Summary & Visualisation